In [172]:
import graphlab

In [173]:
sales = graphlab.SFrame('kc_house_data.gl/')

In [174]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [175]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features] # A numpy matrix whose columns are the desired features plus a constant column (this is how we create an 'intercept')
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output] # A numpy array containing the values of the output
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

For testing let's use the 'sqft_living' feature and a constant as our features and price as our output:

In [176]:
(example_features, example_output) = get_numpy_data(sales, ['sqft_living'], 'price') # the [] around 'sqft_living' makes it a list
print example_features[0,:] # this accesses the first row of the data the ':' indicates 'all columns'
print example_output[0] # and the corresponding output

[  1.00000000e+00   1.18000000e+03]
221900.0


In [177]:
print(example_features)

[[  1.00000000e+00   1.18000000e+03]
 [  1.00000000e+00   2.57000000e+03]
 [  1.00000000e+00   7.70000000e+02]
 ..., 
 [  1.00000000e+00   1.02000000e+03]
 [  1.00000000e+00   1.60000000e+03]
 [  1.00000000e+00   1.02000000e+03]]


### Predicting output given regression weights

In [178]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

In [179]:
my_weights = np.array([1., 1.]) # the example weights
my_features = example_features[0,] # we'll use the first data point
test_predictions = predict_output(example_features, my_weights)
# We are just multiplying the features by 1
print test_predictions[0] # should be 1181.0
print test_predictions[1] # should be 2571.0

1181.0
2571.0


### Computing the Derivative

In [180]:
def feature_derivative(errors, feature):
    # Assume that errors and feature are both numpy arrays of the same length (number of data points)
    # compute twice the dot product of these vectors as 'derivative' and return the value
    derivative = 2*np.dot(errors,feature)
    return(derivative)

In [181]:
# Test feature derivative
(example_features, example_output) = get_numpy_data(sales, ['sqft_living'], 'price') 
my_weights = np.array([0., 0.]) # this makes all the predictions 0
test_predictions = predict_output(example_features, my_weights) 
print ("Test predictions: {}").format(test_predictions)
print ("Example outputs: {}").format(example_output)

# just like SFrames 2 numpy arrays can be elementwise subtracted with '-': 
errors = test_predictions - example_output # prediction errors in this case is just the -example_output
feature = example_features[:,0] # let's compute the derivative with respect to 'constant', the ":" indicates "all rows"
print(errors)
print("Example features: {}").format(example_features)
print(feature)


Test predictions: [ 0.  0.  0. ...,  0.  0.  0.]
Example outputs: [ 221900.  538000.  180000. ...,  402101.  400000.  325000.]
[-221900. -538000. -180000. ..., -402101. -400000. -325000.]
Example features: [[  1.00000000e+00   1.18000000e+03]
 [  1.00000000e+00   2.57000000e+03]
 [  1.00000000e+00   7.70000000e+02]
 ..., 
 [  1.00000000e+00   1.02000000e+03]
 [  1.00000000e+00   1.60000000e+03]
 [  1.00000000e+00   1.02000000e+03]]
[ 1.  1.  1. ...,  1.  1.  1.]


In [182]:
derivative = feature_derivative(errors, feature)
# print (feature)
# print (errors)
print derivative
print -np.sum(example_output)*2 # should be the same as derivative

-23345850022.0
-23345850022.0


### Gradient Descent

In [183]:
from math import sqrt # recall that the magnitude/length of a vector [g[0], g[1], g[2]] is sqrt(g[0]^2 + g[1]^2 + g[2]^2)

In [184]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False 
    weights = np.array(initial_weights) # make sure it's a numpy array
    while not converged:
        # compute the predictions based on feature_matrix and weights using your predict_output() function
        predictions = predict_output(feature_matrix, weights)
        # compute the errors as predictions - output
        errors = predictions - output
        gradient_sum_squares = 0 # initialize the gradient sum of squares
        # while we haven't reached the tolerance yet, update each feature's weight
        for i in range(len(weights)): # loop over each weight
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            # compute the derivative for weight[i]:
            derivative = feature_derivative(errors, feature_matrix[:,1])
            # add the squared value of the derivative to the gradient sum of squares (for assessing convergence)
            gradient_sum_squares += derivative**2
            # subtract the step size times the derivative from the current weight
            weights[i] -= (step_size * derivative)
        # compute the square-root of the gradient sum of squares to get the gradient matnigude:
        gradient_magnitude = sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

### Running the Gradient Descent as Simple Regression

In [185]:
train_data,test_data = sales.random_split(.8,seed=0)

# let's test out the gradient descent
simple_features = ['sqft_living']
my_output = train_data['price']
(simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

In [186]:
print("Feature: {}").format(simple_feature_matrix)
print("Weights: {} ").format(initial_weights)
print(output)

Feature: [[  1.00000000e+00   1.18000000e+03]
 [  1.00000000e+00   2.57000000e+03]
 [  1.00000000e+00   7.70000000e+02]
 ..., 
 [  1.00000000e+00   1.53000000e+03]
 [  1.00000000e+00   1.60000000e+03]
 [  1.00000000e+00   1.02000000e+03]]
Weights: [ -4.70000000e+04   1.00000000e+00] 
[['7129300520' datetime.datetime(2014, 10, 13, 0, 0, tzinfo=GMT +0.0)
  221900.0 ..., 1340.0 5650.0 1]
 ['6414100192' datetime.datetime(2014, 12, 9, 0, 0, tzinfo=GMT +0.0)
  538000.0 ..., 1690.0 7639.0 1]
 ['5631500400' datetime.datetime(2015, 2, 25, 0, 0, tzinfo=GMT +0.0)
  180000.0 ..., 2720.0 8062.0 1]
 ..., 
 ['0263000018' datetime.datetime(2014, 5, 21, 0, 0, tzinfo=GMT +0.0)
  360000.0 ..., 1530.0 1509.0 1]
 ['0291310100' datetime.datetime(2015, 1, 16, 0, 0, tzinfo=GMT +0.0)
  400000.0 ..., 1410.0 1287.0 1]
 ['1523300157' datetime.datetime(2014, 10, 15, 0, 0, tzinfo=GMT +0.0)
  325000.0 ..., 1020.0 1357.0 1]]


### Run gradient descent as simple regression

In [187]:
train_data,test_data = sales.random_split(.8,seed=0)

In [188]:
# let's test out the gradient descent
simple_features = ['sqft_living']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

In [189]:
test_weight = regression_gradient_descent(simple_feature_matrix, train_data[my_output], initial_weights, step_size, tolerance)

In [190]:
print(result)

[-46719.2006469    281.7993531]


#### Quiz Question: What is the value of the weight for sqft_living -- the second element of ‘simple_weights’ (rounded to 1 decimal place)?
Use your newly estimated weights and your predict_output() function to compute the predictions on all the TEST data (you will need to create a numpy array of the test feature_matrix and test output first:

In [191]:
(test_simple_feature_matrix, test_output) = get_numpy_data(test_data, simple_features, my_output)
print(test_simple_feature_matrix)
print(test_output)

[[  1.00000000e+00   1.43000000e+03]
 [  1.00000000e+00   2.95000000e+03]
 [  1.00000000e+00   1.71000000e+03]
 ..., 
 [  1.00000000e+00   2.52000000e+03]
 [  1.00000000e+00   2.31000000e+03]
 [  1.00000000e+00   1.02000000e+03]]
[ 310000.  650000.  233000. ...,  610685.  400000.  402101.]


Now compute your predictions using test_simple_feature_matrix and your weights from above.

In [192]:
test_predictions = predict_output(test_simple_feature_matrix, test_weight)
print (test_predictions)

[ 356253.87428756  784588.89100111  435157.69315585 ...,  663415.16916767
  604237.30501646  240716.13951614]


#### Quiz Question: What is the predicted price for the 1st house in the TEST data set for model 1 (round to nearest dollar)?

In [193]:
print("%0.2f" % test_predictions[0])

356253.87


Now that you have the predictions on test data, compute the RSS on the test data set. Save this value for comparison later. Recall that RSS is the sum of the squared errors (difference between prediction and output).

In [194]:
rss = ((test_output - test_predictions)**2).sum()
print(rss)

2.75393142455e+14


### Multiple Regression

In [195]:
model_features = ['sqft_living', 'sqft_living15'] # sqft_living15 is the average squarefeet for the nearest 15 neighbors. 
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, model_features, my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

Use the above parameters to estimate the model weights. Record these values for your quiz.

In [196]:
mult_test_weight = regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance)
print(mult_test_weight)

[-99840.65143618    160.34856382    160.34856382]


Use your newly estimated weights and the predict_output function to compute the predictions on the TEST data. Don't forget to create a numpy array for these features from the test set first!

In [197]:
(mult_test_feature_matrix, test_output) = get_numpy_data(test_data, model_features, my_output)

mult_test_predictions = predict_output(mult_test_feature_matrix, mult_test_weight)
print(mult_test_predictions)

[ 414878.23841973  716333.53839764  339514.41342525 ...,  708316.11020674
  564002.4027705   227270.41875262]


#### Quiz Question: What is the predicted price for the 1st house in the TEST data set for model 2 (round to nearest dollar)?

In [198]:
print("%0.2f" % mult_test_predictions[0])

414878.24



What is the actual price for the 1st house in the test data set?

In [199]:
print(test_data['price'][0])

310000.0


#### Quiz Question: Which estimate was closer to the true price for the 1st house on the Test data set, model 1 or model 2?

Now use your predictions and the output to compute the RSS for model 2 on TEST data.

In [200]:
print(mult_test_predictions)
print(test_output)
diff = mult_test_predictions - test_output
mult_rss = (diff**2).sum()
print("RSS: %f" % mult_rss)

[ 414878.23841973  716333.53839764  339514.41342525 ...,  708316.11020674
  564002.4027705   227270.41875262]
[ 310000.  650000.  233000. ...,  610685.  400000.  402101.]
RSS: 277538511068266.750000


#### Quiz Question: Which model (1 or 2) has lowest RSS on all of the TEST data?

In [201]:
print(mult_rss)
print(rss)

2.77538511068e+14
2.75393142455e+14
